# Day 3: Multi-GPU Virtual Memory Management

> 🛣️ **"What if different GPUs could access the same memory like reserved highway lanes—you control exactly who can use which lanes and when?"**

## Learning Objectives
By the end of this session, you will be able to:
1. Configure cross-GPU memory access permissions using VMM
2. Compare VMM-based sharing vs traditional peer access
3. Map virtual memory across multiple devices
4. Use fabric handles for large NVLink cluster deployments

## Multi-GPU Memory Sharing

VMM enables fine-grained control over which GPUs can access which memory:

```cpp
// Grant GPU 1 access to memory on GPU 0
CUmemAccessDesc accessDesc;
accessDesc.location.type = CU_MEM_LOCATION_TYPE_DEVICE;
accessDesc.location.id = 1;  // GPU 1 gets access
accessDesc.flags = CU_MEM_ACCESS_FLAGS_PROT_READWRITE;
cuMemSetAccess(ptr_on_gpu0, size, &accessDesc, 1);
```

## 🛣️ Concept Card: Multi-GPU VMM as Reserved Highway Lanes

**The Analogy:** Traditional peer access is like opening all highway lanes to everyone—simple but no control. VMM is like having **reserved lanes** where you control exactly which vehicles (GPUs) can use which sections:

```
🛣️ RESERVED HIGHWAY LANE SYSTEM
═══════════════════════════════════════════════════════════
Traditional P2P Access:          VMM Access Control:
┌─────────────────────┐          ┌─────────────────────────┐
│  🚗 ALL OR NOTHING  │          │  🛣️ RESERVED LANES      │
│                     │          │                         │
│  GPU0 ←→ GPU1       │          │  Lane 1: GPU0 only ═══  │
│  Full access or     │          │  Lane 2: GPU0+GPU1 ═══  │
│  no access          │          │  Lane 3: GPU1 only ═══  │
│                     │          │  Lane 4: All GPUs ═════ │
│                     │          │                         │
│  No per-buffer      │          │  Per-buffer control!    │
│  control            │          │                         │
└─────────────────────┘          └─────────────────────────┘
        ↓                                  ↓
   Binary on/off                   Fine-grained permissions
   for entire GPU                  per memory region!
═══════════════════════════════════════════════════════════
```

**The Lane Reservation System:**
| Highway Concept | VMM Operation | Purpose |
|-----------------|---------------|---------|
| **Reserve lanes** | `cuMemAddressReserve()` | Claim virtual address space |
| **Build road section** | `cuMemCreate()` | Allocate physical memory on specific GPU |
| **Connect to highway** | `cuMemMap()` | Map physical to virtual address |
| **Open lane access** | `cuMemSetAccess()` | Grant R/W to specific GPUs |
| **NVLink interchange** | Fabric handles | High-speed GPU interconnect |

**Why This Matters:** Like lane reservations that let you control traffic flow precisely, VMM lets you control exactly which GPUs access which data—essential for security, performance, and complex multi-GPU algorithms!

In [ ]:
%%writefile multi_gpu_vmm.cu
#include <stdio.h>
#include <cuda.h>
#include <cuda_runtime.h>

__global__ void writeKernel(int* data, int n, int value) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) data[idx] = value;
}

__global__ void readKernel(int* data, int n, long long* sum) {
    __shared__ long long sdata[256];
    int tid = threadIdx.x;
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    
    sdata[tid] = (idx < n) ? data[idx] : 0;
    __syncthreads();
    
    for (int s = 128; s > 0; s >>= 1) {
        if (tid < s) sdata[tid] += sdata[tid + s];
        __syncthreads();
    }
    
    if (tid == 0) atomicAdd(sum, sdata[0]);
}

int main() {
    int deviceCount;
    cudaGetDeviceCount(&deviceCount);
    printf("Found %d GPU(s)\n", deviceCount);
    
    if (deviceCount < 2) {
        printf("Multi-GPU VMM requires 2+ GPUs. Demonstrating single-GPU case.\n");
        
        // Single GPU demonstration
        cuInit(0);
        
        CUmemAllocationProp prop = {};
        prop.type = CU_MEM_ALLOCATION_TYPE_PINNED;
        prop.location.type = CU_MEM_LOCATION_TYPE_DEVICE;
        prop.location.id = 0;
        
        size_t granularity;
        cuMemGetAllocationGranularity(&granularity, &prop,
                                      CU_MEM_ALLOC_GRANULARITY_MINIMUM);
        
        size_t size = 1 << 20;
        size = ((size + granularity - 1) / granularity) * granularity;
        
        CUdeviceptr ptr;
        cuMemAddressReserve(&ptr, size, granularity, 0, 0);
        
        CUmemGenericAllocationHandle handle;
        cuMemCreate(&handle, size, &prop, 0);
        cuMemMap(ptr, size, 0, handle, 0);
        
        CUmemAccessDesc accessDesc = {};
        accessDesc.location.type = CU_MEM_LOCATION_TYPE_DEVICE;
        accessDesc.location.id = 0;
        accessDesc.flags = CU_MEM_ACCESS_FLAGS_PROT_READWRITE;
        cuMemSetAccess(ptr, size, &accessDesc, 1);
        
        int n = size / sizeof(int);
        writeKernel<<<(n+255)/256, 256>>>((int*)ptr, n, 1);
        
        long long* d_sum;
        cudaMalloc(&d_sum, sizeof(long long));
        cudaMemset(d_sum, 0, sizeof(long long));
        readKernel<<<(n+255)/256, 256>>>((int*)ptr, n, d_sum);
        
        long long h_sum;
        cudaMemcpy(&h_sum, d_sum, sizeof(long long), cudaMemcpyDeviceToHost);
        printf("Sum: %lld (expected %d)\n", h_sum, n);
        
        cudaFree(d_sum);
        cuMemUnmap(ptr, size);
        cuMemRelease(handle);
        cuMemAddressFree(ptr, size);
        
        return 0;
    }
    
    // Multi-GPU case
    cuInit(0);
    
    // Check P2P capability
    int canAccess;
    cudaDeviceCanAccessPeer(&canAccess, 1, 0);
    if (!canAccess) {
        printf("GPU 1 cannot access GPU 0 memory\n");
        return 1;
    }
    printf("P2P access supported between GPU 0 and GPU 1\n");
    
    // Allocate on GPU 0 with VMM
    cudaSetDevice(0);
    
    CUmemAllocationProp prop = {};
    prop.type = CU_MEM_ALLOCATION_TYPE_PINNED;
    prop.location.type = CU_MEM_LOCATION_TYPE_DEVICE;
    prop.location.id = 0;
    
    size_t granularity;
    cuMemGetAllocationGranularity(&granularity, &prop,
                                  CU_MEM_ALLOC_GRANULARITY_MINIMUM);
    
    size_t size = 1 << 20;
    size = ((size + granularity - 1) / granularity) * granularity;
    
    CUdeviceptr ptr;
    cuMemAddressReserve(&ptr, size, granularity, 0, 0);
    
    CUmemGenericAllocationHandle handle;
    cuMemCreate(&handle, size, &prop, 0);
    cuMemMap(ptr, size, 0, handle, 0);
    
    // Grant access to BOTH GPUs
    CUmemAccessDesc accessDescs[2];
    for (int i = 0; i < 2; i++) {
        accessDescs[i].location.type = CU_MEM_LOCATION_TYPE_DEVICE;
        accessDescs[i].location.id = i;
        accessDescs[i].flags = CU_MEM_ACCESS_FLAGS_PROT_READWRITE;
    }
    cuMemSetAccess(ptr, size, accessDescs, 2);
    printf("Granted access to GPU 0 and GPU 1\n");
    
    // GPU 0 writes
    cudaSetDevice(0);
    int n = size / sizeof(int);
    writeKernel<<<(n+255)/256, 256>>>((int*)ptr, n, 42);
    cudaDeviceSynchronize();
    printf("GPU 0 wrote data\n");
    
    // GPU 1 reads (P2P access!)
    cudaSetDevice(1);
    long long* d_sum;
    cudaMalloc(&d_sum, sizeof(long long));
    cudaMemset(d_sum, 0, sizeof(long long));
    readKernel<<<(n+255)/256, 256>>>((int*)ptr, n, d_sum);
    
    long long h_sum;
    cudaMemcpy(&h_sum, d_sum, sizeof(long long), cudaMemcpyDeviceToHost);
    printf("GPU 1 read sum: %lld (expected %lld)\n", h_sum, (long long)n * 42);
    
    // Cleanup
    cudaFree(d_sum);
    cuMemUnmap(ptr, size);
    cuMemRelease(handle);
    cuMemAddressFree(ptr, size);
    
    printf("Multi-GPU VMM complete!\n");
    return 0;
}

In [ ]:
!nvcc multi_gpu_vmm.cu -o multi_gpu_vmm -lcuda && ./multi_gpu_vmm

## Fabric Handles for NVLink Clusters

For larger clusters with NVLink/NVSwitch, fabric handles enable efficient memory sharing:

```cpp
// Export allocation as fabric handle
CUmemFabricHandle fabricHandle;
cuMemExportToShareableHandle(&fabricHandle, handle, 
                              CU_MEM_HANDLE_TYPE_FABRIC, 0);

// Import on another GPU
CUmemGenericAllocationHandle importedHandle;
cuMemImportFromShareableHandle(&importedHandle, &fabricHandle,
                                CU_MEM_HANDLE_TYPE_FABRIC);
```

---

## 🎯 Exercises

### 🔷 CUDA C++ Exercises (Primary)

Complete these exercises to practice multi-GPU VMM programming:

In [ ]:
%%writefile multi_gpu_vmm_exercises.cu
#include <stdio.h>
#include <cuda.h>
#include <cuda_runtime.h>

/*
 * Multi-GPU VMM Exercises
 * 
 * Exercise 1: Implement peer access using VMM
 * - Allocate memory on GPU 0 using VMM
 * - Grant GPU 1 read/write access via cuMemSetAccess
 * - Launch kernels on both GPUs accessing same memory
 * 
 * Exercise 2: Ring buffer across GPUs
 * - Create shared memory region accessible by all GPUs
 * - Implement producer-consumer with GPU 0 writing, GPU 1 reading
 * 
 * Exercise 3: Compare P2P methods
 * - cudaEnablePeerAccess vs VMM-based access
 * - Measure bandwidth differences
 */

#define CU_CHECK(call) do { \
    CUresult err = call; \
    if (err != CUDA_SUCCESS) { \
        const char* errStr; \
        cuGetErrorString(err, &errStr); \
        printf("CUDA Driver Error: %s at %s:%d\n", errStr, __FILE__, __LINE__); \
        exit(1); \
    } \
} while(0)

#define CUDA_CHECK(call) do { \
    cudaError_t err = call; \
    if (err != cudaSuccess) { \
        printf("CUDA Error: %s at %s:%d\n", cudaGetErrorString(err), __FILE__, __LINE__); \
        exit(1); \
    } \
} while(0)

__global__ void writeKernel(int* data, int n, int value) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        data[idx] = value + idx;
    }
}

__global__ void readKernel(int* data, int n, long long* sum) {
    __shared__ long long sdata[256];
    int tid = threadIdx.x;
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    
    sdata[tid] = (idx < n) ? data[idx] : 0;
    __syncthreads();
    
    for (int s = 128; s > 0; s >>= 1) {
        if (tid < s) sdata[tid] += sdata[tid + s];
        __syncthreads();
    }
    
    if (tid == 0) atomicAdd(sum, sdata[0]);
}

__global__ void bandwidthKernel(float* src, float* dst, size_t n) {
    size_t idx = blockIdx.x * blockDim.x + threadIdx.x;
    size_t stride = gridDim.x * blockDim.x;
    for (size_t i = idx; i < n; i += stride) {
        dst[i] = src[i];
    }
}

// Exercise 1: VMM-based Peer Access
void exercise1_vmmPeerAccess() {
    printf("=== Exercise 1: VMM-based Peer Access ===\n");
    
    int deviceCount;
    CUDA_CHECK(cudaGetDeviceCount(&deviceCount));
    
    if (deviceCount < 2) {
        printf("This exercise requires 2+ GPUs. Showing single-GPU simulation.\n\n");
        
        // Single GPU case - demonstrate the API calls
        const int N = 1024;
        const size_t size = N * sizeof(int);
        
        cuInit(0);
        
        // Get allocation properties for GPU 0
        CUmemAllocationProp prop = {};
        prop.type = CU_MEM_ALLOCATION_TYPE_PINNED;
        prop.location.type = CU_MEM_LOCATION_TYPE_DEVICE;
        prop.location.id = 0;
        
        size_t granularity;
        CU_CHECK(cuMemGetAllocationGranularity(&granularity, &prop, 
                                                CU_MEM_ALLOC_GRANULARITY_MINIMUM));
        
        printf("Allocation granularity: %zu bytes\n", granularity);
        printf("In multi-GPU case, you would:\n");
        printf("  1. cuMemAddressReserve for VA range\n");
        printf("  2. cuMemCreate for physical memory on GPU 0\n");
        printf("  3. cuMemMap to map physical to VA\n");
        printf("  4. cuMemSetAccess to grant GPU 1 access\n");
        printf("  5. Launch kernels on both GPUs\n\n");
        
        return;
    }
    
    // TODO Exercise 1: Multi-GPU VMM peer access
    printf("Implement VMM-based peer access:\n");
    
    cuInit(0);
    
    const int N = 1 << 20;
    const size_t size = N * sizeof(int);
    
    // Step 1: Get properties for GPU 0
    CUmemAllocationProp prop = {};
    prop.type = CU_MEM_ALLOCATION_TYPE_PINNED;
    prop.location.type = CU_MEM_LOCATION_TYPE_DEVICE;
    prop.location.id = 0;
    
    size_t granularity;
    CU_CHECK(cuMemGetAllocationGranularity(&granularity, &prop, 
                                            CU_MEM_ALLOC_GRANULARITY_MINIMUM));
    
    size_t alignedSize = ((size + granularity - 1) / granularity) * granularity;
    
    // TODO: Reserve VA, create physical memory, map it
    CUdeviceptr ptr;
    CUmemGenericAllocationHandle handle;
    
    // YOUR CODE HERE:
    // CU_CHECK(cuMemAddressReserve(&ptr, alignedSize, granularity, 0, 0));
    // CU_CHECK(cuMemCreate(&handle, alignedSize, &prop, 0));
    // CU_CHECK(cuMemMap(ptr, alignedSize, 0, handle, 0));
    
    // TODO: Set access for GPU 0
    // CUmemAccessDesc accessDesc0 = {};
    // accessDesc0.location = prop.location;
    // accessDesc0.flags = CU_MEM_ACCESS_FLAGS_PROT_READWRITE;
    // CU_CHECK(cuMemSetAccess(ptr, alignedSize, &accessDesc0, 1));
    
    // TODO: Grant access to GPU 1
    // CUmemAccessDesc accessDesc1 = {};
    // accessDesc1.location.type = CU_MEM_LOCATION_TYPE_DEVICE;
    // accessDesc1.location.id = 1;  // GPU 1
    // accessDesc1.flags = CU_MEM_ACCESS_FLAGS_PROT_READWRITE;
    // CU_CHECK(cuMemSetAccess(ptr, alignedSize, &accessDesc1, 1));
    
    printf("VMM peer access setup complete (uncomment code to run)\n\n");
    
    // Cleanup would go here
}

// Exercise 2: Ring Buffer Pattern
void exercise2_ringBuffer() {
    printf("=== Exercise 2: Multi-GPU Ring Buffer ===\n");
    
    int deviceCount;
    CUDA_CHECK(cudaGetDeviceCount(&deviceCount));
    
    printf("Ring buffer pattern for producer-consumer:\n");
    printf("  - GPU 0: Producer (writes data)\n");
    printf("  - GPU 1: Consumer (reads data)\n");
    printf("  - Shared memory region with VMM access control\n\n");
    
    // TODO Exercise 2: Implement ring buffer
    printf("Steps to implement:\n");
    printf("  1. Allocate buffer on GPU 0 with VMM\n");
    printf("  2. Grant GPU 1 read access via cuMemSetAccess\n");
    printf("  3. GPU 0 writes to head, GPU 1 reads from tail\n");
    printf("  4. Use atomic operations for head/tail pointers\n\n");
    
    if (deviceCount < 2) {
        printf("Requires 2+ GPUs for full implementation.\n\n");
        return;
    }
    
    // YOUR CODE HERE for multi-GPU ring buffer
}

// Exercise 3: P2P Method Comparison
void exercise3_p2pComparison() {
    printf("=== Exercise 3: P2P Method Comparison ===\n");
    
    int deviceCount;
    CUDA_CHECK(cudaGetDeviceCount(&deviceCount));
    
    if (deviceCount < 2) {
        printf("Requires 2+ GPUs. Showing expected measurements.\n");
        printf("  cudaEnablePeerAccess: Simple, automatic optimization\n");
        printf("  VMM cuMemSetAccess: Fine-grained control, per-allocation\n");
        printf("  Expected: Similar bandwidth when NVLink available\n\n");
        return;
    }
    
    // TODO Exercise 3: Compare bandwidth
    printf("Implement bandwidth comparison:\n");
    printf("  Method 1: cudaEnablePeerAccess + cudaMalloc\n");
    printf("  Method 2: VMM + cuMemSetAccess\n");
    printf("  Measure: GPU-to-GPU memory copy bandwidth\n\n");
    
    const size_t SIZE = 1ULL << 28;  // 256 MB
    const int ITERATIONS = 10;
    
    // Check P2P capability
    int canAccessPeer;
    CUDA_CHECK(cudaDeviceCanAccessPeer(&canAccessPeer, 0, 1));
    printf("P2P access 0->1: %s\n", canAccessPeer ? "Yes" : "No");
    
    if (!canAccessPeer) {
        printf("P2P not supported between GPUs\n\n");
        return;
    }
    
    // YOUR CODE HERE:
    // 1. Enable peer access with cudaEnablePeerAccess
    // 2. Allocate on GPU 0, access from GPU 1
    // 3. Time the bandwidth
    // 4. Repeat with VMM approach
    // 5. Compare results
    
    printf("Implement timing comparison (uncomment code)\n\n");
}

int main() {
    printf("=== Multi-GPU VMM Exercises ===\n\n");
    
    int deviceCount;
    CUDA_CHECK(cudaGetDeviceCount(&deviceCount));
    printf("Found %d GPU(s)\n\n", deviceCount);
    
    exercise1_vmmPeerAccess();
    exercise2_ringBuffer();
    exercise3_p2pComparison();
    
    printf("=== Exercises Complete ===\n");
    printf("For multi-GPU exercises, run on a system with 2+ GPUs.\n");
    printf("Uncomment TODO sections to complete implementations.\n");
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o multi_gpu_vmm_exercises multi_gpu_vmm_exercises.cu -lcuda && ./multi_gpu_vmm_exercises

### 🔶 Python/Numba Exercises (Optional)

The following exercises explore multi-GPU concepts accessible in Python/Numba:

1. **Multi-GPU detection**: Use `cuda.gpus` to enumerate devices and check `cuda.gpus[i].compute_capability`
2. **Cross-device transfers**: Implement data exchange between GPUs using `cuda.select_device()` and explicit copies
3. **Peer access check**: Use Numba's CUDA API to check peer accessibility between GPU pairs

## Key Takeaways

### 🛣️ Memory Management Patterns Learned

| Pattern | Traditional P2P | VMM Multi-GPU | Benefit |
|---------|----------------|---------------|---------|
| **Selective Sharing** | All-or-nothing | Per-buffer permissions | Security & control |
| **Dynamic Access** | Static peer enable | Runtime access changes | Flexibility |
| **Large Clusters** | Complex setup | Fabric handles | Scalability |

### Core Multi-GPU VMM Concepts

1. **Fine-Grained Access** - Control exactly which GPUs access which memory regions
2. **`cuMemSetAccess`** - Grant read/write permissions per device per allocation
3. **VMM vs P2P** - More control than `cudaEnablePeerAccess`, works per-buffer
4. **Fabric Handles** - Scale to large NVLink/NVSwitch clusters efficiently

### 🛣️ The Reserved Lanes Model
```
Physical Alloc (GPU0) → Map to VA → Grant Access (GPU1) → Cross-GPU Kernel
      🏗️ Build           🔗 Connect    🎫 Lane Pass         🚗 Traffic Flows
```

**Next:** Custom allocators for application-specific memory optimization!